**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 90 Mins**

### Import Libraries

In [21]:
import pandas as pd

### Load Dataset

In [22]:
Data_set = pd.read_csv("dataset.csv")

### EDA

- check NaNs

In [23]:
Data_set.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

- check duplicates

In [24]:
Data_set[["tweet" ,"label"]].duplicated().sum()

2432

- show samples of data texts to find out required preprocessing steps

In [25]:
Data_set.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


- check dataset balancing

In [26]:
Data_set["label"].value_counts()

0    29720
1     2242
Name: label, dtype: int64

- Cleaning and Preprocessing are:
    - 1
    - 2
    - 3

### Cleaning and Preprocessing

In [27]:
duplicated_df = Data_set[Data_set[["tweet" ,"label"]].duplicated()]

In [28]:
Data_without_dup = Data_set.drop(duplicated_df.index)

In [29]:
pd.set_option('display.max_colwidth', 100000)

In [30]:
Data_without_dup.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [31]:
# from sklearn.model_selection import train_test_split
# Train_tweets , Test_tweets , Train_target , Test_target = train_test_split(Data_set["tweet"] 
#                                                                            , Data_set["label"] , 
#                                                                            test_size = 0.2
#                                                                           ,stratify = Data_set["label"]
#                                                                           , random_state = 42)

In [32]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions# load spacy model, can be "en_core_web_sm" as well
nlp = spacy.load('en_core_web_md')

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [33]:
Train_data = Data_without_dup

In [34]:
# nltk.download('stopwords')
stop_words = stopwords.words('english')
for w in stop_words:
    nlp.vocab[w].is_stop = False

In [35]:
import re

In [36]:
def process_tweet(tweet):
    return " ".join(re.sub("([\@|\#][A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

def text_preprocessing(text):
    text = process_tweet(text)
    
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    text = text.strip()
    text =  " ".join(text.split())

    text = unidecode.unidecode(text)

    text = contractions.fix(text)

    text = text.lower()

    doc = nlp(text) 
    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        
        if token.is_stop and token.pos_ != 'NUM': 
            flag = False
        if token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        if token.pos_ == 'SYM' and flag == True: 
            flag = False
        if (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        if token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        elif token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [37]:
Train_tweets = pd.DataFrame(Train_tweets)
Train_target = pd.DataFrame(Train_target)

In [38]:
print(Train_tweets.shape , Train_target.shape)
Train_data = pd.concat([Train_tweets , Train_target] , axis=1)

(25569, 1) (25569, 1)


In [39]:
for i in range(Train_tweets.shape[0]):
    Train_data.iloc[i , 0] = ' '.join(text_preprocessing(Train_data.iloc[i , 0]))

In [40]:
# Train_tweets = Train_tweets.apply(lambd a x:' '.join(text_preprocessing(x)))
# Train_tweets.head()

**If it takes 60 Mins till here, you are doing Great** <br>
**If not! You also are doing Great**

### Modelling

In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn import metrics
import eli5

In [42]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5))
clf = LinearSVC()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(Train_data["tweet"], Train_data["label"])

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5))),
                ('linearsvc', LinearSVC())])

#### Evaluation

In [43]:
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [44]:
print_report(pipe_tfidf, Test_tweets, Test_target)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5945
           1       0.88      0.44      0.59       448

    accuracy                           0.96      6393
   macro avg       0.92      0.72      0.78      6393
weighted avg       0.95      0.96      0.95      6393

accuracy: 0.957


In [45]:
eli5.show_weights(clf, vec=vec, top=20)

C:\Users\Abdelrahman\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+1.298,ism
+1.246,jew
+1.241,jew
+1.116,sexis
+1.082,sexi
+1.064,sexi
+1.060,rac
+1.060,onse
+1.056,uk
+1.048,aci


### Enhancement

- Using different N-grams
- Using different text representation technique

In [46]:
# from sklearn.preprocessing import LabelEncoder
# # from tensorflow.keras.utils import to_categorical

# encoder = LabelEncoder()
# y_encoded = encoder.fit_transform(Data_set['tweet'])
# y_encoded = to_categorical(y_encoded)

In [47]:
def get_longest_text(texts):
    longest_input = 0
    for text in texts:
        text_len= len(text.split())
        longest_input = max(longest_input, text_len)
    return longest_input

In [48]:
longest_input = get_longest_text(Data_set['tweet'])

In [49]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [50]:
from tqdm import tqdm

In [51]:
Train_data = Train_data[0:100]

In [52]:
import numpy as np
data_emb = np.zeros((len(Train_data['tweet']), longest_input, 300))
for i, text in enumerate(tqdm(nlp.pipe(Train_data['tweet']), total=len(Train_data['tweet']))):
    for j, token in enumerate(text):
        data_emb[i, j] = token.vector

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 632.92it/s]


In [53]:
import tensorflow as tf

# define the network
inputs = tf.keras.layers.Input((longest_input, 300))
reshaped = tf.keras.layers.Reshape((longest_input, 300, 1))(inputs)


filters = [2, 3, 4]

# define the conv net
conv_1 = tf.keras.layers.Conv2D(100, (filters[0], 300), activation='relu')(reshaped)
conv_2 = tf.keras.layers.Conv2D(100, (filters[1], 300), activation='relu')(reshaped)
conv_3 = tf.keras.layers.Conv2D(100, (filters[2], 300), activation='relu')(reshaped)

# define max-pooling
pool_1 = tf.keras.layers.MaxPooling2D((longest_input - filters[0] + 1, 1), strides=(1,1))(conv_1)
pool_2 = tf.keras.layers.MaxPooling2D((longest_input - filters[1] + 1, 1), strides=(1,1))(conv_2)
pool_3 = tf.keras.layers.MaxPooling2D((longest_input - filters[2] + 1, 1), strides=(1,1))(conv_3)

# concatenate the convs
merged_tensor = tf.keras.layers.concatenate([pool_1, pool_2, pool_3], axis=1)

# now flatten them and add a dense layer
flatten = tf.keras.layers.Flatten()(merged_tensor)

# add a dense layer
clf = tf.keras.layers.Dense(100, activation='relu')(flatten)

# add final output
clf = tf.keras.layers.Dense(1, activation='sigmoid')(clf)

In [54]:
model = tf.keras.models.Model(inputs, clf)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 34, 300)]    0           []                               
                                                                                                  
 reshape (Reshape)              (None, 34, 300, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 33, 1, 100)   60100       ['reshape[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 1, 100)   90100       ['reshape[0][0]']                
                                                                                              

In [55]:
!pip install graphviz 

In [56]:
tf.keras.utils.plot_model(
    model, show_shapes=False, show_layer_names=True,
    rankdir='TB', expand_nested=False, dpi=96
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [57]:
Train_data

,tweet,label
26247,do my order at black amp sexy from s collection,0
13681,it s there it s I m tell that andrew jackson be and,1
25676,video on have have of this crap,1
14544,after monaco a podium this time guy maybe,0
25411,wow open amateur hour on fox just the golf and be people walk in of commentator on air,0
...,...,...
23171,tom and jerry icepole I be feel well now,0
6524,man rory didn t up,0
26,all when be in sunday love,0
10404,a surprise delivery this morning a gorgeous thank you gift from a lovely client,0


In [58]:
print(Train_tweets["tweet"] , Train_target["label"])

26247                             did my first order at @user ð ... something black &amp; sexy from @user 's collection!   #cantwait  
13681                       @user @user ð it's there, it's @user ðalthough i'm told that andrew jackson was  and #cantankerous also
25676                                            @user video: @user on #hateful  #liberals "#america has had enough of this crap!"  @user
14544                                             @user @user after monaco a podium this time guys maybe? #mclarenhonda #canadiangp   #f1
25411    @user @user wow. us open amateur hour on fox. just show the golf and be quite! people walking in front of commentators on air?  
                                                                       ...                                                               
15438         @user scratch my last tweet, jo, sorry. someone had "borrowed" two boxes without telling me. we have the full complement!  
29797                             

In [59]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_emb , Train_data["label"] , test_size=.2)

In [60]:
# train our model
model.fit(x_train, y_train)

3/3 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - acc: 0.8125


In [61]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 177ms/step - loss: 0.0000e+00 - acc: 0.8500


[0.0, 0.8500000238418579]

#### Done!